# Modelling of physical systems

## First Assignment

### Miłosz Góralczyk

The implemented physical models describes the effects of Advection-Diffusion Equation, describes the transport of a scalar property, such as temperature or salinity, through a combination of advection and diffusion processes in a fluid medium.

The model can be presented by an equation:

$$
\frac{\partial c}{\partial t} + U \frac{\partial c}{\partial x} - D \frac{\partial^2 c}{\partial^2 x} = 0
$$

$\frac{\partial c}{\partial t}$​ Time Derivative - Represents the rate of change of concentration over time.

$U \frac{\partial c}{\partial x}$ Advection Term - Describes the transport of concentration due to a uniform velocity *U*.

$D \frac{\partial^2 c}{\partial^2 x}$​ Diffusion Term - Models the spreading of the substance due to diffusion, governed by the diffusivity *D*.

The sum of time evolution, advection, and diffusion effects must balance. ($ = 0$)

The equation itself was already modelled on the previous laboratory, and it has been proven to correctly model the phenomena.

In this assignment, I'm going to model how the *Intensity* of the observed concentration - that being heat, solvent, an odor - changes over time *in one observed place*

Basically, it's going to answer the question - if we placed a stationary sensor in the distance X on the path of the transported property - how would its reading look like over time?

The equation itself:

$$
c(x, t) = \frac{M}{A}\frac{1}{2\sqrt{\pi D}}\frac{x}{t^\frac{3}{2}}e^{\Bigg( -\frac{(ut - x)^2}{4Dt} \Bigg)}
$$

models the movement of flow of a substance through an one-dimentional medium. The meaningful symbols here being:

Constants
- $M$ - mass of the substance, unit $[kg]$
- $A$ - volumetric flow rate, unit $[\frac{m^3}{s}]$
- $D$ - diffusion coefficient, unit $[\frac{m^2}{s}]$
- $u$ - speed of the medium, unit $[\frac{m}{s}]$

Variables
- $x$ - lenght of the medium, unit $[m]$
- $t$ - time passed, unit $[s]$

giving us finally...

- $c(x, t)$ - concentration of the property at a given point at a given time, unit $[\frac{kg}{m^3}]$,




I could assume that here, what is being modelled is a distribution over time of some isotope in the air, moving to the right because of the wind, while also diffusing itself slowly.

The isotope is detected at some positions by a stationary sensors, that save the readings of the observed concentration as a continous time/value pairs

Of course the assumption is, all the mass of the isotope remains on the 1D space - no substance leaks out to the right/left.

## The Code

In [ ]:
# Imports

import pint
import numpy as np
import scipy
from matplotlib import pyplot
import open_atmos_jupyter_utils

In [ ]:
# pint si unit initialization

si = pint.UnitRegistry()
si.setup_matplotlib()

In [ ]:
# Function to define concentration distribution

def concentration_function(mass, area, diffusivity, velocity):
    def c(x, t) :
        return (
            (mass * x * np.exp(-((velocity * t - x) ** 2) / (4 * diffusivity * t)))
            / (2 * area * (np.pi * diffusivity) ** 0.5 * t ** 1.5)
        )
    return c

In [ ]:
# Function to generate concentration function with fixed x value

def concentration_at_x(c_func, x: np.ndarray):
    return lambda t: c_func(x, t)

In [ ]:
# Define constants with appropriate units

MASS = np.float64(5.3) * si.kg
AREA = np.float64(5) * si.m ** -3 / si.s
DIFFUSIVITY = np.float64(0.00003) * si.m ** 2 / si.s
VELOCITY = np.float64(0.0056) * si.m / si.s


In [ ]:
# Generate time array

TIME = np.linspace(0.1, 100.1, 1000, dtype=np.float64) * si.s

In [ ]:
# Create the fixed position

X_POS1 = np.float64(0.15) * si.m
X_POS2 = np.float64(0.35) * si.m

In [ ]:
# Compute concentration values for two fixed positions

concentration = concentration_function(MASS, AREA, DIFFUSIVITY, VELOCITY)

concentration_fixed_x_1 = concentration_at_x(concentration, X_POS1)
concentration_result_1 = concentration_fixed_x_1(TIME)

concentration_fixed_x_2 = concentration_at_x(concentration, X_POS2)
concentration_result_2 = concentration_fixed_x_2(TIME)

In [ ]:
# Plot the first concentration result

fig = pyplot.figure(figsize=(6, 3.5))
fig.gca().plot(TIME, concentration_result_1)
fig.suptitle(f'Concentration of isotope at X1={X_POS1.magnitude}m over time')
fig.gca().grid()
open_atmos_jupyter_utils.show_plot(fig=fig)

In [ ]:
# Plot the second concentration result

fig = pyplot.figure(figsize=(6, 3.5))
fig.gca().plot(TIME, concentration_result_2)
fig.suptitle(f'Concentration of isotope at X2={X_POS2.magnitude}m over time')
fig.gca().grid()
open_atmos_jupyter_utils.show_plot(fig=fig)

### The results

- As it can be seen, the two graphs, at x1 = 0.15m and x2 = 0.35m logically represent what we might have initially thought.

- The closer to the initial point location - x1 - observed the property first - picking up readings at around 13s, peaking at 24s, and losing readings at around 60s

- The farther one - x2 - picked up at 33s, peaked around a minute (60s), with positive readings going a little over the 100s

- further down the line, the curve was also more spread out, as a result of the diffusion part of the equation, while the curve at x1 was narrower

- Because of that diffusion, the maximum observed value was higher at sensor x1 ($0.64kg/m^3$), than later at sensor x2 ($0.40kg/m^3$), when the substance was more "spread out"

- overall, the amount of total mass observed is the same in both cases, so the area under the curve should be the same.

- The shape of the graph resembles a bell curve to some degree, although it is not an exact linear normal distribution, as the "sides" of the bell are not symmetric

- Nevertheless, if one was to compare the dependencies of mean and variance of those graphs as with the normal distribution, one could say that as the mean increases, the variance increases too. 